In [1]:
#Utilities

import pandas as pd
import numpy as np


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 



import random
from collections import Counter
from sklearn.metrics import roc_curve, auc, average_precision_score

pd.set_option('display.max_columns', 100)



Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
user_df = pd.read_csv('../mod5-capstone/data/Kaggle/steam-200k.csv',
                  header= None, 
                  names = ['userid', 'game', 'behavior', 'hours', 'other'])

steam_df = pd.read_csv('games.csv')

In [3]:
steam_df = steam_df.loc[:, ~steam_df.columns.str.contains('^Unnamed')]

In [4]:
user_df.head()

,userid,game,behavior,hours,other
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


In [5]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 5 columns):
userid      200000 non-null int64
game        200000 non-null object
behavior    200000 non-null object
hours       200000 non-null float64
other       200000 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.6+ MB


In [6]:
# clean behavior df
user_df['hours_played'] = user_df['hours'].astype('float32')
user_df.loc[(user_df['behavior'] == 'purchase') & (user_df['hours'] == 1.0), 'hours_played'] = 0

In [7]:
user_df.sort_values(['userid', 'game', 'hours_played'], inplace = True)
user_df.head()

,userid,game,behavior,hours,other,hours_played
65429,5250,Alien Swarm,purchase,1.0,0,0.0
65430,5250,Alien Swarm,play,4.9,0,4.9
65423,5250,Cities Skylines,purchase,1.0,0,0.0
65424,5250,Cities Skylines,play,144.0,0,144.0
65435,5250,Counter-Strike,purchase,1.0,0,0.0


In [8]:
user_df.drop_duplicates(['userid', 'game'], keep = 'last', inplace = True)


In [9]:
user_df.head()

,userid,game,behavior,hours,other,hours_played
65430,5250,Alien Swarm,play,4.9,0,4.9
65424,5250,Cities Skylines,play,144.0,0,144.0
65435,5250,Counter-Strike,purchase,1.0,0,0.0
65436,5250,Counter-Strike Source,purchase,1.0,0,0.0
65437,5250,Day of Defeat,purchase,1.0,0,0.0


In [10]:
user_df.drop(['behavior', 'hours', 'other'], axis = 1, inplace = True)
user_df.head()

,userid,game,hours_played
65430,5250,Alien Swarm,4.9
65424,5250,Cities Skylines,144.0
65435,5250,Counter-Strike,0.0
65436,5250,Counter-Strike Source,0.0
65437,5250,Day of Defeat,0.0


In [11]:
user_df.to_csv('user_df.csv')

In [12]:
n_users = len(user_df.userid.unique())
n_games = len(user_df.game.unique())

print('There are {0} users and {1} games in the dataset'.format(n_users, n_games))

There are 12393 users and 5155 games in the dataset


In [13]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128804 entries, 65430 to 129085
Data columns (total 3 columns):
userid          128804 non-null int64
game            128804 non-null object
hours_played    128804 non-null float32
dtypes: float32(1), int64(1), object(1)
memory usage: 3.4+ MB


In [14]:
steam_df

,appid,name,release_date,developer,publisher,required_age,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,rating_ratio,rating,release_year,mods,multi_player,online_co_op,online_multi_player,single_player,action,adventure,casual,early_access,indie,massively_multiplayer,rpg,racing,simulation,sports,strategy,p_metacritic,type
0,10,Counter-Strike,2000-11-01,Valve,Valve,0,Action,0,159745,4173,21211,172,10000000,9.99,163918,0.974542,96.141982,2000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,85.423725,Paid
1,20,Team Fortress Classic,1999-04-01,Valve,Valve,0,Action,0,4335,771,798,11,2000000,4.99,5106,0.849001,81.652169,1999,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,76.355907,Paid
2,30,Day of Defeat,2003-05-01,Valve,Valve,0,Action,0,4325,482,894,18,5000000,4.99,4807,0.899730,86.467857,2003,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,78.662717,Paid
3,40,Deathmatch Classic,2001-06-01,Valve,Valve,0,Action,0,1592,345,1200,8,5000000,4.99,1937,0.821890,77.979832,2001,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,74.989380,Paid
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,0,Action,0,8907,477,1316,281,5000000,4.99,9384,0.949169,91.893508,1999,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,82.020638,Paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33315,1376940,Monster Killer,2020-08-05,Laush Dmitriy Sergeevich,Laush Studio,0,Casual;Indie,4,1,0,0,0,0,0.99,1,1.000000,59.416365,2020,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,66.306482,Paid
33316,1377140,Don't Look,2020-08-07,Diego Aguilar,Diego Aguilar,0,Adventure;Indie,3,0,2,0,0,0,4.99,2,0.000000,0.000000,2020,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,50.732637,Paid
33317,1377900,Train Train Train,2020-08-07,heinn,heinn,0,Casual;Indie;Simulation;Strategy,18,2,0,0,0,0,4.99,2,1.000000,64.079515,2020,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,67.814440,Paid
33318,1378290,The Citadel,2020-08-05,doekuramori,doekuramori,0,Action;Casual;Indie,0,53,4,0,0,0,14.99,57,0.929825,79.288583,2020,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,72.577810,Paid


In [15]:
steam_df['merge_name'] = steam_df['name'].str.replace(':','')

In [16]:
steam_df.drop(['genres'], axis = 1, inplace = True)

In [17]:
steam_df.head()

,appid,name,release_date,developer,publisher,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,rating_ratio,rating,release_year,mods,multi_player,online_co_op,online_multi_player,single_player,action,adventure,casual,early_access,indie,massively_multiplayer,rpg,racing,simulation,sports,strategy,p_metacritic,type,merge_name
0,10,Counter-Strike,2000-11-01,Valve,Valve,0,0,159745,4173,21211,172,10000000,9.99,163918,0.974542,96.141982,2000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,85.423725,Paid,Counter-Strike
1,20,Team Fortress Classic,1999-04-01,Valve,Valve,0,0,4335,771,798,11,2000000,4.99,5106,0.849001,81.652169,1999,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,76.355907,Paid,Team Fortress Classic
2,30,Day of Defeat,2003-05-01,Valve,Valve,0,0,4325,482,894,18,5000000,4.99,4807,0.899730,86.467857,2003,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,78.662717,Paid,Day of Defeat
3,40,Deathmatch Classic,2001-06-01,Valve,Valve,0,0,1592,345,1200,8,5000000,4.99,1937,0.821890,77.979832,2001,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,74.989380,Paid,Deathmatch Classic
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,0,0,8907,477,1316,281,5000000,4.99,9384,0.949169,91.893508,1999,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,82.020638,Paid,Half-Life Opposing Force


In [18]:
merged_df = pd.merge(left = user_df, right= steam_df, how='left', left_on= 'game', right_on = 'merge_name')


In [19]:
merged_df.dropna(axis=0, how='any', inplace=True)


In [20]:
n_users = len(merged_df.userid.unique())
n_games = len(merged_df.game.unique())

print('There are {0} users and {1} games in the dataset'.format(n_users, n_games))

There are 10821 users and 2726 games in the dataset


In [21]:
merged_df.drop(['merge_name'], axis= 1, inplace= True)

In [22]:
dummy = pd.get_dummies(merged_df['type'])
merged_df = pd.concat([merged_df, dummy], axis = 1)
merged_df.drop(['type'], axis= 1, inplace= True)

In [23]:
merged_df.head()

,userid,game,hours_played,appid,name,release_date,developer,publisher,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,rating_ratio,rating,release_year,mods,multi_player,online_co_op,online_multi_player,single_player,action,adventure,casual,early_access,indie,massively_multiplayer,rpg,racing,simulation,sports,strategy,p_metacritic,Free,Paid
0,5250,Alien Swarm,4.9,630.0,Alien Swarm,2010-07-19,Valve,Valve,0.0,66.0,18226.0,994.0,310.0,84.0,1000000.0,0.00,19220.0,0.948283,92.394066,2010.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.383988,1,0
1,5250,Cities Skylines,144.0,255710.0,Cities: Skylines,2015-03-10,Colossal Order Ltd.,Paradox Interactive,0.0,111.0,119378.0,9667.0,2090.0,666.0,5000000.0,29.99,129045.0,0.925088,91.170169,2015.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,82.670419,0,1
2,5250,Counter-Strike,0.0,10.0,Counter-Strike,2000-11-01,Valve,Valve,0.0,0.0,159745.0,4173.0,21211.0,172.0,10000000.0,9.99,163918.0,0.974542,96.141982,2000.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.423725,0,1
3,5250,Counter-Strike Source,0.0,240.0,Counter-Strike: Source,2004-11-01,Valve,Valve,0.0,147.0,105394.0,4359.0,9135.0,415.0,10000000.0,9.99,109753.0,0.960284,94.569366,2004.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.926946,0,1
4,5250,Day of Defeat,0.0,30.0,Day of Defeat,2003-05-01,Valve,Valve,0.0,0.0,4325.0,482.0,894.0,18.0,5000000.0,4.99,4807.0,0.899730,86.467857,2003.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.662717,0,1


In [24]:
merged_df.isnull().sum()

userid                   0
game                     0
hours_played             0
appid                    0
name                     0
release_date             0
developer                0
publisher                0
required_age             0
achievements             0
positive_ratings         0
negative_ratings         0
average_playtime         0
median_playtime          0
owners                   0
price                    0
total_ratings            0
rating_ratio             0
rating                   0
release_year             0
mods                     0
multi_player             0
online_co_op             0
online_multi_player      0
single_player            0
action                   0
adventure                0
casual                   0
early_access             0
indie                    0
massively_multiplayer    0
rpg                      0
racing                   0
simulation               0
sports                   0
strategy                 0
p_metacritic             0
F

In [25]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74706 entries, 0 to 128844
Data columns (total 39 columns):
userid                   74706 non-null int64
game                     74706 non-null object
hours_played             74706 non-null float32
appid                    74706 non-null float64
name                     74706 non-null object
release_date             74706 non-null object
developer                74706 non-null object
publisher                74706 non-null object
required_age             74706 non-null float64
achievements             74706 non-null float64
positive_ratings         74706 non-null float64
negative_ratings         74706 non-null float64
average_playtime         74706 non-null float64
median_playtime          74706 non-null float64
owners                   74706 non-null float64
price                    74706 non-null float64
total_ratings            74706 non-null float64
rating_ratio             74706 non-null float64
rating                   74706 no

In [26]:
merged_df.to_csv('data_for_recommendation_engine.csv')

In [27]:
df = pd.read_csv('data_for_recommendation_engine.csv')

In [28]:
n_users = len(df.userid.unique())
n_games = len(df.game.unique())

print('There are {0} users and {1} games in the dataset'.format(n_users, n_games))

There are 10821 users and 2726 games in the dataset


In [29]:
sparsity = df.shape[0] /float(n_users * n_games)
print('{:.2%} of the user-item matrix is filled'.format(sparsity))

0.25% of the user-item matrix is filled


In [30]:
user_counter = Counter()
for user in df.userid.tolist():
    user_counter[user] +=1

game_counter = Counter()
for game in df.game.tolist():
    game_counter[game] += 1

    
user2idx = {user: i for i, user in enumerate(df.userid.unique())}
idx2user = {i: user for user, i in user2idx.items()}

game2idx = {game: i for i, game in enumerate(df.game.unique())}
idx2game = {i: game for game, i in game2idx.items()}

In [31]:
user_idx = df['userid'].apply(lambda x: user2idx[x]).values
game_idx = df['gameidx'] = df['game'].apply(lambda x: game2idx[x]).values
hours = df['hours_played'].values

# EXP
metascore = df['p_metacritic'].values

In [32]:
#Create Matrix

In [33]:
#Start by creating a zero matrix with the shape of number of users by number of games
zero_matrix = np.zeros(shape = (n_users, n_games))

#Create preferences matrix with whether or not a user has purchased a game
user_game_pref = zero_matrix.copy()
user_game_pref[user_idx, game_idx] = 1

#Create a confidence matrix with number of hours user has

user_game_interactions = zero_matrix.copy()
user_game_interactions[user_idx, game_idx] = hours +1


# Experimental matrix with predicted metascores for games

user_game_scores = zero_matrix.copy()

user_game_scores[user_idx, game_idx] = metascore 

When it comes to recommender systems, we are interested in recommending a certain number of items to a user. So it makes sense to compare precision within the n number of items rather than the whole lot. Therefore, we'll set a user-defined level of k in order to evaluate the model. 

In [34]:
k = 3 # User defined value
ktimes = 2

purchase_counts = np.apply_along_axis(np.bincount, 1, user_game_pref.astype(int)) # Count number of purchases per user

buyers_idx = np.where(purchase_counts[:,1]>=ktimes *k)[0] # Find number of users who purchase 2*k games

print('{0} users bought {1} or more games'.format(len(buyers_idx), (ktimes*k)))

2420 users bought 6 or more games


In [35]:
test_frac = 0.2 # Set aside 10% for validation and another 10% for testing

test_users_idx = np.random.choice(buyers_idx,
                                  size= int(np.ceil(len(buyers_idx))* test_frac),
                                  replace = False)


In [36]:
val_users_idx = test_users_idx[:int(len(test_users_idx) / 2)]

test_users_idx = test_users_idx[int(len(test_users_idx) /2):]

In [37]:
# A function used to mask the preferences data from training matrix
def data_process(dat, train, test, user_idx, k):
    for user in user_idx:
        purchases = np.where(dat[user, :] == 1)[0]
        mask = np.random.choice(purchases, size = k, replace = False)
        train[user, mask] = 0
        test[user, mask] = dat[user, mask]
    return train, test

In [38]:
train_matrix = user_game_pref.copy()
test_matrix = zero_matrix.copy()
val_matrix = zero_matrix.copy()


# Mask the train matrix and create the validation and test matrices
train_matrix, val_matrix = data_process(user_game_pref, train_matrix, val_matrix, val_users_idx, k)
train_matrix, test_matrix = data_process(user_game_pref, train_matrix, test_matrix, test_users_idx, k)

In [39]:
test_matrix[test_users_idx[0], test_matrix[test_users_idx[0], :].nonzero()[0]]

array([1., 1., 1.])

In [40]:
train_matrix[test_users_idx[0], test_matrix[test_users_idx[0], :].nonzero()[0]]

array([0., 0., 0.])

In [41]:
tf.reset_default_graph() #Create a new graph

pref =tf.placeholder(tf.float32, (n_users, n_games)) # Preference matrix

interactions = tf.placeholder(tf.float32, (n_users, n_games)) #Hours played

p_metascore = tf.placeholder(tf.float32, (n_users, n_games)) # predicted metascore

In [42]:
n_features = 30 #number of latent features to be extracted

# user latent preferences with a shape of user x latent features

X = tf.Variable(tf.truncated_normal([n_users, n_features], mean = 0, stddev = 0.05))

# game latent features with a shape of game x latent features
Y = tf.Variable(tf.truncated_normal([n_games, n_features], mean = 0, stddev = 0.05))

#Confidence parameter
conf_alpha = tf.Variable(tf.random_uniform([1], 0, 1))

In [43]:
# Initialize a user bias vector
user_bias = tf.Variable(tf.truncated_normal([n_users, 1], stddev = 0.2))


# Concatenate the vector to the user matrix
# Due to how matrix algebra works, we also need to add a column of ones to make sure
# the resulting calculation will take into account the item biases.

X_plus_bias = tf.concat([X, 
                         user_bias,
                         tf.ones((n_users, 1), dtype = tf.float32)], axis = 1)

In [44]:
# Initialize the item bias vector
item_bias = tf.Variable(tf.truncated_normal([n_games, 1], stddev = 0.2))

# Cocatenate the vector to the game matrix
# Also, adds a column one for the same reason stated above.
Y_plus_bias = tf.concat([Y, 
                         tf.ones((n_games, 1), dtype = tf.float32),
                         item_bias],
                         axis = 1)


In [45]:
# Here, we finally multiply the matrices together to estimate the predicted preferences
pred_pref = tf.matmul(X_plus_bias, Y_plus_bias, transpose_b=True)
# Construct the confidence matrix with the hours played and alpha paramter
conf = 1 + conf_alpha * interactions



In [46]:
cost = tf.reduce_sum(tf.multiply(conf, tf.square(tf.subtract(pref, pred_pref))))

l2_sqr = tf.nn.l2_loss(X) + tf.nn.l2_loss(Y) + tf.nn.l2_loss(user_bias) + tf.nn.l2_loss(item_bias)

lambda_c = 0.01

loss = cost + lambda_c * l2_sqr

In [47]:
lr = 0.05
optimize = tf.train.AdagradOptimizer(learning_rate= lr).minimize(loss)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [48]:
# This is a function that helps to calculate the top k precision 
def top_k_precision(pred, mat, k, user_idx):
    precisions = []
    
    for user in user_idx:
        rec = np.argsort(-pred[user, :]) # Found the top recommendation from the predictions
        
        top_k = rec[:k]
        
        labels = mat[user, :].nonzero()[0]
        
        precision = len(set(top_k) & set(labels)) / float(k) # Calculate the precisions from actual labels
        precisions.append(precision)
    
    return np.mean(precisions) 

In [49]:
iterations = 80
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(iterations):
        sess.run(optimize, feed_dict = {pref: train_matrix,
                                        interactions: user_game_interactions})
        if i % 10 == 0:
            mod_loss = sess.run(loss, feed_dict = {pref: train_matrix,
                                                   interactions: user_game_interactions})            
            mod_pred = pred_pref.eval()
            train_precision = top_k_precision(mod_pred, train_matrix, k, val_users_idx)
            val_precision = top_k_precision(mod_pred, val_matrix, k, val_users_idx)
            print('Iterations {0}...'.format(i),
                  'Training Loss {:.2f}...'.format(mod_loss),
                  'Train Precision {:.3f}...'.format(train_precision),
                  'Val Precision {:.3f}'.format(val_precision)
                )
    rec = pred_pref.eval()
    test_precision = top_k_precision(rec, test_matrix, k, test_users_idx)
    print('\n')
    print('Test Precision{:.3f}'.format(test_precision))

Iterations 0... Training Loss 1873173.38... Train Precision 0.091... Val Precision 0.008
Iterations 10... Training Loss 135553.00... Train Precision 0.431... Val Precision 0.052
Iterations 20... Training Loss 101821.58... Train Precision 0.521... Val Precision 0.045
Iterations 30... Training Loss 87580.52... Train Precision 0.601... Val Precision 0.054
Iterations 40... Training Loss 77948.45... Train Precision 0.665... Val Precision 0.050
Iterations 50... Training Loss 70405.55... Train Precision 0.698... Val Precision 0.055
Iterations 60... Training Loss 63917.27... Train Precision 0.725... Val Precision 0.059
Iterations 70... Training Loss 57826.07... Train Precision 0.738... Val Precision 0.069


Test Precision0.062


In [50]:
n_examples = 5
users = np.random.choice(test_users_idx, size = n_examples, replace = False)
rec_games = np.argsort(-rec)

In [51]:
for user in users:
    print('Recommended Games for {0} are ...'.format(idx2user[user]))
    purchase_history = np.where(train_matrix[user, :] != 0)[0]
    recommendations = rec_games[user, :]
    new_recommendations = recommendations[~np.in1d(recommendations, purchase_history)][:k]
    print('We recommend these games')
    print(', '.join([idx2game[game] for game in new_recommendations]))
    print('\n')
    print('The games that the user actually purchased are ...')
    print(', '.join([idx2game[game] for game in np.where(test_matrix[user, :] != 0)[0]]))
    print('\n')
    print('Precision of {0}'.format(len(set(new_recommendations) & set(np.where(test_matrix[user, :] != 0)[0])) / float(k)))
    print('--------------------------------------')
    print('\n')

Recommended Games for 110881546 are ...
We recommend these games
Clicker Heroes, Counter-Strike Global Offensive, Euro Truck Simulator 2


The games that the user actually purchased are ...
Portal 2, DmC Devil May Cry, Dust An Elysian Tail


Precision of 0.0
--------------------------------------


Recommended Games for 13556140 are ...
We recommend these games
Counter-Strike Condition Zero, Half-Life, Team Fortress Classic


The games that the user actually purchased are ...
Counter-Strike Source, Half-Life, Team Fortress Classic


Precision of 0.6666666666666666
--------------------------------------


Recommended Games for 98624880 are ...
We recommend these games
Deathmatch Classic, Counter-Strike, Half-Life


The games that the user actually purchased are ...
Counter-Strike, Deathmatch Classic, Half-Life


Precision of 1.0
--------------------------------------


Recommended Games for 4318904 are ...
We recommend these games
Day of Defeat, Deathmatch Classic, Counter-Strike Condit

In [52]:
n_examples = 10
users = np.random.choice(test_users_idx, size = n_examples, replace = False)
rec_games = np.argsort(-rec)

In [53]:
for user in users:
    print('Recommended Games for {0} are ...'.format(idx2user[user]))
    purchase_history = np.where(train_matrix[user, :] != 0)[0]
    recommendations = rec_games[user, :]
    new_recommendations = recommendations[~np.in1d(recommendations, purchase_history)][:k]
    print('We recommend these games')
    print(', '.join([idx2game[game] for game in new_recommendations]))
    print('\n')
    print('The games that the user actually purchased are ...')
    print(', '.join([idx2game[game] for game in np.where(test_matrix[user, :] != 0)[0]]))
    print('\n')
    print('Precision of {0}'.format(len(set(new_recommendations) & set(np.where(test_matrix[user, :] != 0)[0])) / float(k)))
    print('--------------------------------------')
    print('\n')

Recommended Games for 16574348 are ...
We recommend these games
Day of Defeat Source, Counter-Strike Global Offensive, Half-Life 2


The games that the user actually purchased are ...
Half-Life 2, Team Fortress 2, Guns of Icarus Online


Precision of 0.3333333333333333
--------------------------------------


Recommended Games for 242937979 are ...
We recommend these games
Portal 2, Counter-Strike Global Offensive, BioShock Infinite


The games that the user actually purchased are ...
Portal 2, Starbound, Terraria


Precision of 0.3333333333333333
--------------------------------------


Recommended Games for 82436907 are ...
We recommend these games
Garry's Mod, Unturned, Trove


The games that the user actually purchased are ...
Defiance, Warframe, TERA


Precision of 0.0
--------------------------------------


Recommended Games for 122620165 are ...
We recommend these games
Terraria, Counter-Strike Condition Zero, The Elder Scrolls V Skyrim


The games that the user actually purcha